In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np
from tqdm import tqdm
import random

# --- Configuration ---
NUM_MODELS = 1000
BATCH_SIZE = 64
NUM_EPOCHS = 1  # A small number of epochs for quick training of many models
LEARNING_RATE = 0.001

# Define the classes for each subset
CLASS1_LABELS = [0, 1, 2, 3]
CLASS2_LABELS = [2, 3, 4, 5]

# Define the "out-of-class" labels for evaluation
# For models trained on CLASS1_LABELS, evaluate on these from CLASS2_LABELS
OOC_CLASS1_EVAL_LABELS = [4, 5]
# For models trained on CLASS2_LABELS, evaluate on these from CLASS1_LABELS
OOC_CLASS2_EVAL_LABELS = [0, 1]

# Set device to GPU if available, otherwise CPU
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")


Using device: cpu


In [40]:

# Set random seeds for reproducibility (for dataset splitting, model init, etc.)
# Note: Training 1000 models means there will still be inherent randomness
# due to different initializations and data shuffling.
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# --- Data Loading and Preprocessing ---
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)) # MNIST mean and std
])

# Download and load the MNIST dataset
print("Loading MNIST dataset...")
full_train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
full_test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)
print("MNIST dataset loaded.")


Loading MNIST dataset...
MNIST dataset loaded.


In [47]:

# --- Dataset Subset Creation Function ---
def create_subset(dataset, labels_to_include):
    """
    Filters a dataset to include only samples with specified labels.
    """
    indices = []
    for i, (_, label) in enumerate(dataset):
        if label in labels_to_include:
            indices.append(i)
    return Subset(dataset, indices)

# Create the specific subsets
print("Creating dataset subsets...")
class1_train_dataset = create_subset(full_train_dataset, CLASS1_LABELS)
class1_test_dataset = create_subset(full_test_dataset, CLASS1_LABELS)

class2_train_dataset = create_subset(full_train_dataset, CLASS2_LABELS)
class2_test_dataset = create_subset(full_test_dataset, CLASS2_LABELS)

# Create out-of-class evaluation datasets
ooc_class1_eval_dataset = create_subset(full_test_dataset, OOC_CLASS1_EVAL_LABELS)
ooc_class2_eval_dataset = create_subset(full_test_dataset, OOC_CLASS2_EVAL_LABELS)
print("Dataset subsets created.")

# --- Neural Network Definition ---
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        # Input layer: 28*28 = 784 pixels
        # Single Hidden layer: 8 neurons (kept from previous layer 1)
        # Output layer: 10 neurons (for digits 0-9)
        self.fc1 = nn.Linear(28 * 28, 10) # Input to the single hidden layer
        # self.relu1 = nn.ReLU()
        # self.fc3 = nn.Linear(8, 10) # Direct connection from hidden layer to output

    def forward(self, x):
        x = x.view(-1, 28 * 28) # Flatten the 28x28 image
        x = self.fc1(x)
        # x = self.relu1(x)
        # x = self.fc3(x)
        return x

# --- Training Function ---
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    """Trains a single neural network model."""
    model.train()
    for epoch in range(num_epochs):
        for data, target in train_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

# --- Evaluation Function ---
def evaluate_model(model, data_loader, description=""):
    """Evaluates a single neural network model and returns accuracy."""
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# --- Main Experiment Logic ---
def run_experiment(train_dataset, self_test_dataset, cross_test_dataset,
                   train_description, self_test_description, cross_test_description):
    """
    Runs the training and evaluation for a given set of datasets.
    Returns a list of trained model state_dicts.
    """
    train_accuracies = []
    self_test_accuracies = []
    cross_test_accuracies = []
    trained_model_weights = [] # List to store model weights

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    self_test_loader = DataLoader(self_test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    cross_test_loader = DataLoader(cross_test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    print(f"\n--- Training {NUM_MODELS} models for {train_description} ---")
    for i in tqdm(range(NUM_MODELS), desc=f"Training models for {train_description}"):
        model = SimpleNN().to(DEVICE)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

        # Train the model
        train_model(model, train_loader, criterion, optimizer, NUM_EPOCHS)

        # Save the model's state_dict (weights)
        trained_model_weights.append(model.state_dict())

        # Evaluate accuracies
        train_acc = evaluate_model(model, train_loader, f"{train_description} Training")
        self_test_acc = evaluate_model(model, self_test_loader, f"{self_test_description} Testing")
        cross_test_acc = evaluate_model(model, cross_test_loader, f"{cross_test_description} Testing")

        train_accuracies.append(train_acc)
        self_test_accuracies.append(self_test_acc)
        cross_test_accuracies.append(cross_test_acc)

        # Print accuracy after each run
        print(f"  Model {i+1}/{NUM_MODELS}:")
        print(f"    Training Acc ({train_description}): {train_acc:.2f}%")
        print(f"    Testing Acc ({self_test_description}): {self_test_acc:.2f}%")
        print(f"    Testing Acc ({cross_test_description}): {cross_test_acc:.2f}%")

    # Calculate and print average results
    avg_train_acc = np.mean(train_accuracies)
    std_train_acc = np.std(train_accuracies)
    avg_self_test_acc = np.mean(self_test_accuracies)
    std_self_test_acc = np.std(self_test_accuracies)
    avg_cross_test_acc = np.mean(cross_test_accuracies)
    std_cross_test_acc = np.std(cross_test_accuracies)

    print(f"\n--- Results for models trained on {train_description} ---")
    print(f"Average Training Accuracy ({train_description}): {avg_train_acc:.2f}% (Std Dev: {std_train_acc:.2f})")
    print(f"Average Testing Accuracy ({self_test_description}): {avg_self_test_acc:.2f}% (Std Dev: {std_self_test_acc:.2f})")
    print(f"Average Testing Accuracy ({cross_test_description}): {avg_cross_test_acc:.2f}% (Std Dev: {std_cross_test_acc:.2f})")
    print("-" * 50)

    return trained_model_weights # Return the list of weights


Creating dataset subsets...
Dataset subsets created.


In [49]:
# Run the experiment for Class 1 models
class1_model_weights = run_experiment(class1_train_dataset, class1_test_dataset, class2_test_dataset,
                                      f"Class 1 (Digits {CLASS1_LABELS})",
                                      f"Class 1 Test (Digits {CLASS1_LABELS})",
                                      f"Class 2 Test (Digits {CLASS2_LABELS})")

# Run the experiment for Class 2 models
class2_model_weights = run_experiment(class2_train_dataset, class2_test_dataset, class1_test_dataset,
                                      f"Class 2 (Digits {CLASS2_LABELS})",
                                      f"Class 2 Test (Digits {CLASS2_LABELS})",
                                      f"Class 1 Test (Digits {CLASS1_LABELS})")

print("\nExperiment complete!")
print(f"Class 1 model weights stored in 'class1_model_weights' (list of {len(class1_model_weights)} state_dicts).")
print(f"Class 2 model weights stored in 'class2_model_weights' (list of {len(class2_model_weights)} state_dicts).")



--- Training 1000 models for Class 1 (Digits [0, 1, 2, 3]) ---


Training models for Class 1 (Digits [0, 1, 2, 3]):   0%|          | 1/1000 [00:02<36:11,  2.17s/it]

  Model 1/1000:
    Training Acc (Class 1 (Digits [0, 1, 2, 3])): 96.73%
    Testing Acc (Class 1 Test (Digits [0, 1, 2, 3])): 97.40%
    Testing Acc (Class 2 Test (Digits [2, 3, 4, 5])): 49.59%


Training models for Class 1 (Digits [0, 1, 2, 3]):   0%|          | 2/1000 [00:04<35:36,  2.14s/it]

  Model 2/1000:
    Training Acc (Class 1 (Digits [0, 1, 2, 3])): 96.81%
    Testing Acc (Class 1 Test (Digits [0, 1, 2, 3])): 97.47%
    Testing Acc (Class 2 Test (Digits [2, 3, 4, 5])): 49.82%


...

  Model 998/1000:
    Training Acc (Class 2 (Digits [2, 3, 4, 5])): 94.38%
    Testing Acc (Class 2 Test (Digits [2, 3, 4, 5])): 94.97%
    Testing Acc (Class 1 Test (Digits [0, 1, 2, 3])): 46.36%


Training models for Class 2 (Digits [2, 3, 4, 5]): 100%|█████████▉| 999/1000 [1:38:49<00:02,  2.08s/it]

  Model 999/1000:
    Training Acc (Class 2 (Digits [2, 3, 4, 5])): 94.51%
    Testing Acc (Class 2 Test (Digits [2, 3, 4, 5])): 94.74%
    Testing Acc (Class 1 Test (Digits [0, 1, 2, 3])): 46.21%


Training models for Class 2 (Digits [2, 3, 4, 5]): 100%|██████████| 1000/1000 [1:38:51<00:00,  5.93s/it]

  Model 1000/1000:
    Training Acc (Class 2 (Digits [2, 3, 4, 5])): 94.68%
    Testing Acc (Class 2 Test (Digits [2, 3, 4, 5])): 94.82%
    Testing Acc (Class 1 Test (Digits [0, 1, 2, 3])): 46.45%

--- Results for models trained on Class 2 (Digits [2, 3, 4, 5]) ---
Average Training Accuracy (Class 2 (Digits [2, 3, 4, 5])): 94.42% (Std Dev: 0.15)
Average Testing Accuracy (Class 2 Test (Digits [2, 3, 4, 5])): 94.71% (Std Dev: 0.23)
Average Testing Accuracy (Class 1 Test (Digits [0, 1, 2, 3])): 46.25% (Std Dev: 0.30)
--------------------------------------------------

Experiment complete!
Class 1 model weights stored in 'class1_model_weights' (list of 1000 state_dicts).
Class 2 model weights stored in 'class2_model_weights' (list of 1000 state_dicts).


In [ ]:
import os
import torch
# Define a directory to save the models
SAVE_DIR = './trained_models_smallest'
os.makedirs(SAVE_DIR, exist_ok=True) # Create the directory if it doesn't exist

# ... (your provided code snippet for running experiments) ...

# Save the weights for Class 1 models
class1_save_path = os.path.join(SAVE_DIR, 'class1_models_weights.pt')
torch.save(class1_model_weights, class1_save_path)
print(f"Saved Class 1 model weights to {class1_save_path}")

# Save the weights for Class 2 models
class2_save_path = os.path.join(SAVE_DIR, 'class2_models_weights.pt')
torch.save(class2_model_weights, class2_save_path)
print(f"Saved Class 2 model weights to {class2_save_path}")

Saved Class 1 model weights to ./trained_models/class1_models_weights.pt
Saved Class 2 model weights to ./trained_models/class2_models_weights.pt


In [44]:
import os
# These should ideally come from your previous run of the 'mnist-subset-nn-experiment' Canvas.
# Load the saved model weights
SAVE_DIR = './trained_models' # Ensure this matches the save directory
class1_save_path = os.path.join(SAVE_DIR, 'class1_models_weights.pt')
class2_save_path = os.path.join(SAVE_DIR, 'class2_models_weights.pt')

class1_model_weights = []
class2_model_weights = []

try:
    if os.path.exists(class1_save_path):
        class1_model_weights = torch.load(class1_save_path, map_location=DEVICE)
        print(f"Loaded {len(class1_model_weights)} Class 1 model state_dicts for fine-tuning.")
    else:
        print(f"Warning: Class 1 model weights file not found at {class1_save_path}. Starting fine-tuning from scratch for Class 1 models.")

    if os.path.exists(class2_save_path): # Although class2_model_weights isn't used in this specific fine-tuning, good to load it if needed later.
        class2_model_weights = torch.load(class2_save_path, map_location=DEVICE)
        print(f"Loaded {len(class2_model_weights)} Class 2 model state_dicts.")
    else:
        print(f"Warning: Class 2 model weights file not found at {class2_save_path}.")

except Exception as e:
    print(f"An error occurred while loading pre-trained model weights: {e}")
    print("Proceeding with empty lists for model weights, which means models will train from scratch.")


# ... (rest of the "Mega Model Fine-Tuning and Evaluation" Canvas code) ...

Loaded 1000 Class 1 model state_dicts for fine-tuning.
Loaded 1000 Class 2 model state_dicts.


/var/folders/_q/vn2v0d2s0lbfbzzf3_rkcf2h0000gn/T/ipykernel_83651/3924068203.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  class1_model_weights = torch.load(class1_sav